In [1]:
# replace the standard sqlite3 module with pysqlite3
# for compatibility with Chroma
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


import langchain
import os
import bs4
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# load environment variables
load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# credential json not required if you are working within vertex AI
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/LLM-agent-with-Gemini/fleet-anagram-244304-7dafcc771b2f.json"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # only if you are using text embedding model from google

In [7]:
from langchain_core.tools import tool
from langchain_core.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

print(calculator.invoke({"a": 2, "b": 3}))
print(calculator.name)
print(calculator.description)
print(calculator.args)

6
Calculator
multiply numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


In [102]:
look_up_dictionary = {
    "Map viewer for detecting defects":"https://www.python.org/",
    "time series trends":"http://python.langchain.com"
}

In [113]:
tools = [lookup]
llm = ChatVertexAI(model="gemini-1.5-pro")

In [112]:
from fuzzywuzzy import process
class KeywordInput(BaseModel):
    k: str = Field(description="key of the dictionary")

def keyword_to_url(k:str)->str:
    best_match = process.extractOne(k, look_up_dictionary.keys())
    if best_match:
        return look_up_dictionary.get(best_match[0], "URL not found")
    else:
        return "URL not found"

lookup = StructuredTool.from_function(
    func=keyword_to_url,
    name="KeyworkdLookUp",
    description="Infer the keywords from user input and use it to match against the keys that has the highest relevance in the dictionary and retrieve the URL associated",
    args_schema=KeywordInput,
    return_direct=True,
)


# print(calculator.invoke({"a": 2, "b": 3}))
print(lookup.name)
print(lookup.description)
print(lookup.args)

KeyworkdLookUp
Infer the keywords from user input and use it to match against the keys that has the highest relevance in the dictionary and retrieve the URL associated
{'k': {'title': 'K', 'description': 'key of the dictionary', 'type': 'string'}}


In [88]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [114]:
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [115]:
agent_executor.invoke({"input": "I want to view the defects of B16C"})



> Entering new AgentExecutor chain...


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..


Question: I want to view the defects of B16C
Thought: I need to find the URL related to defects of B16C.
Action: KeyworkdLookUp
Action Input: defects of B16Chttps://www.python.org/


> Finished chain.


{'input': 'I want to view the defects of B16C',
 'output': 'https://www.python.org/'}

In [116]:
prompt_template_payload = """
Task: Fill up the API payload based on the user input


Payload Structure:
{
    "dsid": (string, examples: "B47R","B16C","Y42M"),
    "lot": (int, examples: 1952291, 1126911,2023161),
    "pid":(string, examples:"FPP","FQQP","FPC")
}

User Input:
{user input}


Instructions:
1. Extract the dsid, lot, pid from the user input.
2. Construct a valid API payload according to the provided structure
"""

In [117]:
tools = []
agent = create_react_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "B47R, 1952291, FQQP"})



> Entering new AgentExecutor chain...
Question: B47R, 1952291, FQQP
Thought: I need more information to understand the pattern in the given sequence. There's a combination of letters and numbers, and it's unclear how they relate. 
Final Answer: I cannot answer this question. I need more context or information about the sequence to identify a pattern and provide the next element. 


> Finished chain.


{'input': 'B47R, 1952291, FQQP',
 'output': 'I cannot answer this question. I need more context or information about the sequence to identify a pattern and provide the next element.'}

In [78]:

agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)


response = agent_executor.invoke(
    {"messages": [HumanMessage(content="I want to view the wafer map, use lookup")]}
)
response["messages"]

[HumanMessage(content='I want to view the wafer map, use lookup', id='caf9d967-3a87-4671-8fbe-1fa4742cf2e5'),
 AIMessage(content='Could you please provide the keyword for the wafer map? \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 44, 'candidates_token_count': 13, 'total_token_count': 57}}, id='run-348342eb-6056-40d6-86a7-e799c113715b-0')]

In [ ]:
search = TavilySearchResults(max_results=3)
result = search.invoke("What is Julia language known for?")

# Formatting
for i, item in enumerate(result, start=1):
    print(f"{i}. URL: {item['url']}\nContent: {item['content']}\n")

In [ ]:
web_paths = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/Rguide.html",
    # "https://google.github.io/styleguide/cppguide.html",
    # "https://google.github.io/styleguide/go/",
    # "https://google.github.io/styleguide/javaguide.html",
    # "https://google.github.io/styleguide/jsguide.html",
    # "https://google.github.io/styleguide/tsguide.html"
]

# Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

docs = []
for path in web_paths:
    loader = WebBaseLoader(web_paths=(path,),
                        #    bs_kwargs={"parse_only": bs4_strainer}
    )
    docs += loader.load()

In [ ]:
len(docs[0].page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
all_splits[-1].metadata

In [ ]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "programming_style_guide_search",
    "Reference for information about styling rules for programming languages",
)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [ ]:
import json

def print_formatted_response(response):
    for i in range(4):
        if i == 0:
            print(f"Question: {response['messages'][0].content}")
        elif i == 1:
            function_call = response['messages'][i].additional_kwargs.get('function_call', {})
            name = function_call.get('name', '')
            print(f"Tool: {name}")
        elif i == 2:
            data = json.loads(response['messages'][2].content)
            print("Citation:")
            for item in data:
                print(item['url'])
        else:
            print(f"Answer: {response['messages'][3].content}")
        print()

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What are the differences between langchain and llamaindex, search if it's not in retriever")]}
)
response["messages"]

In [ ]:
print(response["messages"][-1].content)

In [ ]:
# streamin
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is Vajra")]}
):
    print(chunk)
    print("----")

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, I'm Hanyu")]}, config
):
    print(chunk)
    print()

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's my name?")]}, config
):
    print(chunk)
    print()

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Suggest some use cases for LLM Agent")]}, config
)

print(response["messages"][-1].content)
    